In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
class MultiLayerPerceptron:

    def __init__(self, input_size, hidden_size, output_size, lr, l2_lambda):
        self.W1 = np.random.randn(input_size, hidden_size) * np.sqrt(6/input_size)
        self.B1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * np.sqrt(6/input_size)
        self.B2 = np.zeros((1, output_size))
        self.lr = lr
        self.l2_lambda = l2_lambda

    def relu(self, x):
        return np.maximum(0, x)

    def relu_derivative(self, x):
        return np.where(x > 0, 1, 0)

    def sigmoid(self, x):
        return 1/(1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def forwardPropagation(self, X):
        self.z1 = np.dot(X, self.W1) + self.B1
        self.a1 = self.relu(self.z1)
        self.z2 = np.dot(self.a1, self.W2) + self.B2
        self.a2 = self.sigmoid(self.z2)
        return self.a2
        
    def backPropagation(self, X, y):
        y = y.reshape(-1, 1)
        
        delta2 = self.a2 - y
        delta1 = np.dot(delta2, self.W2.T) * self.relu_derivative(self.a1)

        self.W2 -= self.lr * (np.dot(self.a1.T, delta2) + self.l2_lambda * self.W2)
        self.B2 -= self.lr * np.sum(delta2, axis=0, keepdims=True)
        self.W1 -= self.lr * (np.dot(X.T, delta1) + self.l2_lambda * self.W1)
        self.B1 -= self.lr * np.sum(delta1, axis=0, keepdims=True)

    def evaluate(self, y, y_hat):
        epsilon = 1e-10
        return -np.mean(y * np.log(y_hat + epsilon) + (1 - y) * np.log(1 - y_hat + epsilon))

    def train(self, X, y, epochs):
        for epoch in range(epochs):
            output = self.forwardPropagation(X)
            self.backPropagation(X, y)

            if epoch % 500 == 0:
                loss = self.evaluate(y, output)
                print(f"Epoch {epoch}, Loss: {loss:.6f}")

In [3]:
df = pd.read_csv('Diabetes Binary Classification.csv')
X = df.iloc[:,0:8].values
y = df.iloc[:, -1].values

In [4]:
X.shape

(768, 8)

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X,y, random_state=2, test_size=0.2)

In [6]:
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)

In [7]:
from sklearn.metrics import accuracy_score

In [8]:
mlp = MultiLayerPerceptron(input_size=8, hidden_size=40, output_size=1, lr=0.001, l2_lambda=0.5)
mlp.train(X_train_scaled, Y_train, epochs=1000)
# Evaluate on test data
y_pred = mlp.forwardPropagation(X_test_scaled)
y_pred_binary = (y_pred >= 0.5).astype(int)

accuracy = np.mean(y_pred_binary == Y_test.reshape(-1, 1))
print(f"\nAccuracy: {accuracy:.4f}")

Epoch 0, Loss: 2.231157
Epoch 500, Loss: 1.175188

Accuracy: 0.7857
